In [ ]:
import numpy as np
from scipy.stats import entropy
import json
import collections
import itertools

import powerlaw
from jointpdf.jointpdf import JointProbabilityMatrix
from jointpdf.jointpdf import FullNestedArrayOfProbabilities

from probability_distributions import JointProbabilityMatrixExtended
import probability_distributions
from probability_distributions import ProbabilityArray
from simulate import find_mean_std_mse
import nudge

In [ ]:
def nudge_distribution_local_non_causal_assume_independence(joint, nudge_label, nudge_size):
    """
    Nudge the marginal and assume independence after the nudge to find the
    new joint.
    
    Parameters:
    ----------
    joint: a numpy array
        Representing a discrete probability distribution
    nudge_label: an integer
    nudge_size: a (small) number
    number_of_nudges: an integer
    
    """
    other_variables_labels = set(range(len(joint.shape))) - set([nudge_label]) 
    marginal_nudge_variable = ProbabilityArray(joint).marginalize(nudge_label)
    marginal_other_variables = ProbabilityArray(joint).marginalize(other_variables_labels)
    marginal_variable_nudged, nudged_states = nudge.nudge(marginal_nudge_variable, nudge_size)
    return probability_distributions.compute_joint_from_independent_marginals(
        marginal_other_variables, marginal_variable_nudged, sorted(list(label_nudged_variable))
    )

In [ ]:
pdf = JointProbabilityMatrix(2, 5, 'random')
#pdf.append_variables_with_target_mi(1, 0.5)
#pdf.append_synergistic_variables()

import numpy as np

probability_array_tryout = FullNestedArrayOfProbabilities(
    np.array(
      [
        [
          [
            [0.2, 0.1]          
          ],
          [
            [0.05, 0.05]
          ]
        ],
        [
          [
            [0.3, 0.05] 
          ],
          [
            [0.15, 0.1] 
          ]
        ],
      ]
    )
) 

In [ ]:
NUDGE_SIZE = 0.01
pdf = JointProbabilityMatrix(1, 6, 'random')
#print(pdf.joint_probabilities.joint_probabilities)
pdf.append_variables_with_target_mi(1, 0.1)
#print(pdf.mutual_information([0], [1]))

joint_old = pdf.joint_probabilities.joint_probabilities
probability_array_old = ProbabilityArray(joint_old)
marginal_variable_old = probability_array_old.marginalize(set([0]))
marginal_function_old = probability_array_old.marginalize(set([1]))
conditional_joint_old, marginal_labels_old, conditional_labels_old = (
    probability_array_old.find_conditional(set([1]), set([0]))
)
marginal_variable_nudged, nudged_states = nudge.nudge(marginal_variable_old, NUDGE_SIZE)
joint_new = ProbabilityArray(probability_distributions.compute_joint(
    marginal_variable_nudged, conditional_joint_old, conditional_labels_old
))

#This takes the KL-divergence between the new and old function variable
marginal_function_new = joint_new.marginalize(set([1]))  

kl_variable = entropy(marginal_variable_old, marginal_variable_nudged)
kl_function = entropy(marginal_function_old, marginal_function_new) 
print("KL-divergence old and new function distribution: {}".format(kl_variable))
print("KL-divergence old and new function distribution: {}".format(kl_function))

In [ ]:
#this method does ot work, the joint does not sum to 1!!!
def update_joint_independent_marginals(joint, label_marginal, marginal, 
                                       update_states, other_marginal):
    """update a joint distribution only for certain states of a marginal
    assuming the merginals are independent for those states
    
    Parameters:
    ----------
    joint: numpy array
    label_marginal: integer
    marginal: a numpy array
    states: a numpy array
    other_marginal: a numpy array
    
    Returns: The updated joint distribution

    """
    updated_joint = np.copy(joint)
    states_input_variables = [range(states) for states in joint.shape]
    states_input_variables[label_marginal] = list(update_states)
    
    for state in itertools.product(*states_input_variables):
        state_other_input_variables = tuple([index for count, index in enumerate(state)
                                             if count != label_marginal])
        updated_joint[state] = (other_marginal[state_other_input_variables] *
                                marginal_variable_nudged[state[label_marginal]])

    return updated_joint



In [ ]:
import numpy as np

tryout = np.arange(2**10).reshape([2]*10)
#print(tryout)
a = [1, 3, 4, 5, 9]
total_variables = 10
b = np.moveaxis(tryout, a, range(total_variables-len(a), total_variables, 1))
c = np.array([[1,10],[100,1000]])
b = b*c
#tryout = np.moveaxis(b, range(total_variables-len(a), total_variables, 1), a)
print(tryout)


#tryout = b*c
#np.moveaxis(tryout, range(total_variables-len(a), total_variables, 1), a)


In [ ]:
def nudge_distribution_local_non_causal(joint, nudge_label, nudge_size, number_of_nudges):
    """
    nudge the the variable with nudge label while keeping the 
    marginal of the other variables constant
    
    Parameters:
    ----------
    joint: a numpy array
        Representing a discrete probability distribution
    nudge_label: an integer
    nudge_size: a (small) number
    number_of_nudges: an integer
    
    Returns: a numpy array, representing the nudged probability distribution
    
    """
    nudged_joint = np.copy(joint)
    nudged_joint = nudged_joint.swapaxes(nudge_label, len(joint.shape)-1)
    nudge_states = nudge.select_random_states(nudged_joint.shape[:-1], number_of_nudges) 
    
    nudged_states_marginal = np.random.choice(joint.shape[nudge_label], 2, replace=False)
    nudge_state_plus, nudge_state_minus = nudged_states_marginal[0], nudged_states_marginal[1]   
    for state in nudge_states:
        plus_state = tuple(copy.copy(state) + [nudge_state_plus])
        minus_state = tuple(copy.copy(state) + [nudge_state_minus])        
        size = min(nudged_joint[minus_state], 1-nudged_joint[plus_state], nudge_size)
        nudged_joint[plus_state] += size
        nudged_joint[minus_state] -= size
    
    nudged_joint = nudged_joint.swapaxes(nudge_label, len(joint.shape)-1)
    return nudged_joint
    
def impact_nudge_causal_output(distribution, function_indices, new_input_distribution):
    """
    Calculate the impact of a nudge of the input distribution on the output. 
    Assuming the output is causally determined using using the input.
    
    Parameters:
    ----------
    distribution: a ProbabilityArray object
    function_indices: a set of integers
    new_input_distribution: a numpy array
        It represents the input distribution after the nudge
    
    Returns:
    -------
    A numpy array representing a probability distribution
    
    """
    variable_indices = set(range(len(distribution.probability_distribution.shape))) - function_indices
    marginal_output_old = distribution.marginalize(function_indices)
    conditional, marginal_labels, conditional_labels = (
        distribution.find_conditional(function_indices, variable_indices)
    )
    distribution_new = ProbabilityArray(probability_distributions.compute_joint(
        new_input_distribution, conditional, conditional_labels
    ))
    marginal_output_new = distribution_new.marginalize(function_indices)  
    kl_divergence = entropy(marginal_output_old, marginal_output_new) 
    return kl_divergence

In [ ]:
def conditional_distribution(self, selected_indices, conditional_indices):
    """create the conditional distribution for the selected_indices given 
    the conditional_indices for the joint_distribution
    
    Parameters:
    ----------
    joint_distribution: numpy array
    selected_indices: list of integers
    conditional_indices: list of integers
    
    Returns:
    -------
    
    """
    joint_distribution = self.marginalize_distribution(selected_indices+conditional_indices)
    marginal_conditional = self.marginalize_distribution(conditional_indices)
    conditional_distribution = np.copy(joint_distribution) 
    it = np.iter(joint_distribution, flags='multi_index')
    while not it.finished:
        conditional_arguments = tuple([it.multi_index[i] for i in conditional_indices])
        conditional_distribution[it.multi_index] = (
            conditional_distribution[it.multi_index] /
            marginal_conditional[conditional_arguments]
        )
        it.iternext()
        
    return conditional_distribution
        

In [ ]:
import numpy as np
import time

a = np.random.random(10**7)
start1 = time.time()
for i in range(100):
    b = np.log2(a)

print(time.time()-start1)
    
start2 = time.time()
for i in range(100):
    b = np.log(a)

print(time.time()-start2)
    
    
    

In [ ]:
def decrease_entropy(distribution, state1, state2, max_difference):
    """
    Decrease the entropy of the distribution randomly

    Parameters:
    ----------
    distribution: a 1-d numpy array 
        Representing a probability distribution
    state1, state2: integers in the range len(distribution)
    
    Returns: a float
    -------
    By how much the entropy was decreased

    """
    if distribution[state1]==0 or distribution[state2]==0:
        return 0
    elif distribution[state1] >= distribution[state2]:
        initial_entropy = np.dot(np.log2(distribution[[state1, state2]]),
                                 distribution[[state1, state2]])
        change = np.random.uniform(0, min(max_difference, distribution[state2]))
        if distribution[state2] < (10**(-10)):
            change = distribution[state2]

        distribution[state1] += change
        distribution[state2] -= change
        if distribution[state2] != 0:
            entropy_after = np.dot(np.log2(distribution[[state1, state2]]), 
                                   distribution[[state1, state2]])
        else:
            entropy_after = np.log2(distribution[state1]) * distribution[state1]
        return -(initial_entropy - entropy_after)
    else:
        return decrease_entropy(distribution, state2, state1, max_difference)



In [ ]:
from scipy.stats import entropy

total_number_of_states = 5**8
a = np.full(total_number_of_states, 1.0/total_number_of_states)
b = np.full(total_number_of_states, 1.0/total_number_of_states)
max_difference = 1.0/total_number_of_states
final_entropy_size = np.log2(total_number_of_states * 0.1)
print("the final_entropy size is: {}".format(final_entropy_size))

#start1 = time.time()

#entropy_size = entropy(a, base=2)
#count = 0
#while entropy_size > final_entropy_size:
#    count += 1
#    state1, state2 = tuple(np.random.choice(total_number_of_states, 2, False))
#    entropy_size -= decrease_entropy(a, state1, state2, max_difference)
#print("the count is {}".format(count))
#print("entropy {}".format(entropy(a, base=2)))

#print(time.time()-start1)


start1 = time.time()

entropy_size = entropy(b, base=2)
while entropy_size > final_entropy_size:
    print('starting again')
    random_positions = np.random.choice(total_number_of_states, total_number_of_states*50*2)
    for i in range(total_number_of_states*50):
        entropy_size -= decrease_entropy(b, random_positions[i], random_positions[i*2], max_difference)
        if entropy_size < final_entropy_size:
            break
    
print("entropy {}".format(entropy(b, base=2)))
print(time.time()-start1)

In [ ]:
import numpy as np

a = np.array([1,2,3,4])
for i, j in enumerate(a):
    print(i, j)

In [ ]:
import numpy as np

a = np.array([[1,2,3,4], [5,6,7,8]])
print(a)
print(np.delete(a, [0], 0))


In [ ]:
import numpy as np

a = np.array([3,2,1,5,1])
a[np.array([False, False, True, True, False])].shape[0]

In [ ]:
import numpy as np

nudge_size = 0.3
a = np.array(
    [
        [
            [0.4, 0.2, 0.4],
            [0.3, 0.5, 0.2]
        ],
        [
            [0.3, 0.3, 0.4],
            [0.2, 0.2, 0.6]
        ]
    ]
)
cond_output = np.array(
    [
        [
            [0.4, 0.2, 0.4],
            [0.3, 0.5, 0.2]
        ],
        [
            [0.3, 0.3, 0.4],
            [0.2, 0.2, 0.6]
        ]
    ]
)


output_state = [1, -1, 1]
input_arr = np.array([0.4, 0.1, 0.1, 0.4])
b = cond_output.flatten() * np.array(output_state*input_arr.shape[0])
c = np.reshape(b, (input_arr.shape[0], len(output_state)))
cond_allignment = np.sum(c, axis=1)
print(cond_allignment)

#input_arr = np.array([0.4, 0.1, 0.1, 0.4])
#output_state = np.array([[1, -1, 1]])
#a1 = np.repeat(output_state, a.flatten().shape[0]/3, axis=0).flatten()
#b = a.flatten() * a1
#c = np.reshape(b, a.shape)
#cond_allignment = np.sum(c, axis=len(a.shape)-1).flatten()
#print(cond_allignment)

#can (probably) be done more efficient by building a tree right away, for now good enough
sorted_result = sorted(
    [[input_item, cond_score, input_var] 
     for input_item, cond_score, input_var 
     in zip(input_arr, cond_allignment, range(input_arr.shape[0]))],
    key=lambda x: x[1]
)
print(sorted_result)

count = 0
minus_weight = 0
while minus_weight < nudge_size and count < len(sorted_result)-1:
    print(count)
    print(sorted_result[count][0])
    minus_weight += sorted_result[count][0]
    count += 1
    
print("the minus weight is {}".format(minus_weight))
print("the count is {}".format(count))
    
if minus_weight < nudge_size:
    #cannot find a big enough nudge
    positive_nudge_impact = sorted_result[-1][1]*minus_weight 
    negative_nudge_impact = sum(
        [nudge*nudge_impact for nudge, nudge_impact, _ in sorted_result[:count-1]]
    ) 
    nudge = [[input_var, -weight] for input_var, weight, _, input_var in sorted_result[:count]]
    nudge.append(sorted_result[-1][2], minus_weight)
else:
    positive_nudge_impact = sorted_result[-1][1]*nudge_size
    last_minus_nudge = nudge_size - sum(item[0] for item in sorted_result[:count-1])
    print([nudge*nudge_impact for nudge, nudge_impact, _ in sorted_result[:count-1]])
    negative_nudge_impact = (
        sum([nudge*nudge_impact for nudge, nudge_impact, _ in sorted_result[:count-1]]) +
        last_minus_nudge * sorted_result[count-1][1]
    )
    nudge = [[input_var, -nudge] for nudge, _, input_var in sorted_result[:count-1]]
    nudge.append([sorted_result[count-1][2], -last_minus_nudge])
    nudge.append([sorted_result[-1][2], nudge_size])

print("positive nudge impact {}".format(positive_nudge_impact))
print("negative nudge impact {}".format(negative_nudge_impact))

print("the nudge is {}".format(nudge))

nudge_impact = positive_nudge_impact - negative_nudge_impact
    
print(nudge_impact)
    

In [ ]:
import numpy as np
import itertools

cond_output = np.array(
    [
        [
            [
                [0.4, 0.2, 0.4],
                [0.3, 0.5, 0.2]
            ],
            [
                [0.25, 0.4, 0.35],
                [0.1, 0.3, 0.6]
            ]
        ],
        [
            [
                [0.3, 0.3, 0.4],
                [0.2, 0.2, 0.6]
            ],
            [
                [0.4, 0.2, 0.4],
                [0.35, 0.35, 0.3]
            ]
        ]
    ]
)
input_dist = np.array(
    [
        [
            [
                0.2,
                0.1
            ],
            [
                0.05,
                0.01
            ]
        ],
        [
            [
                0.3,
                0.04
            ],
            [
                0.06,
                0.24
            ]
        ]
    ]
)
nudge_states = [1]
nudge_size = 0.01

#start algorithm
number_of_input_states = input_arr.flatten().shape[0]
input_arr = np.moveaxis(input_dist, nudge_states, 
                        list(range(len(nudge_states))))
scores = np.moveaxis(cond_output, nudge_states, 
                     list(range(len(nudge_states))))
weighted_scores = np.multiply(
    np.repeat(input_arr.flatten(), cond_output.shape[-1]),
    scores.flatten()
)
print("")
print("weighted_scores")
print(np.moveaxis(np.reshape(weighted_scores, cond_output.shape), nudge_states, list(range(len(nudge_states)))))

weights = np.sum(
    input_arr,
    axis=tuple(range(len(nudge_states), len(input_arr.shape), 1))
)

print("weights")
print(weights)
print("")

#print("summing over the right axis")
#print(scores.shape)
#print(tuple(range(len(nudge_states), len(input_dist.shape), 1)))

max_impact = 0
max_nudge = []
all_allignments_output_states = itertools.product([-1, 1], repeat=cond_output.shape[-1])
for allignment_output_state in all_allignments_output_states:
    all_the_same = (all([i==1 for i in allignment_output_state]) or 
                    all([i==-1 for i in allignment_output_state]))
    if all_the_same:
        continue
        
    print(allignment_output_state)
    allignment = weighted_scores * np.array(allignment_output_state*number_of_input_states)
    allignment_scores = np.sum(
        np.reshape(allignment, cond_output.shape),
        axis=tuple(range(len(nudge_states), len(cond_output.shape), 1))
    )
    #print("allignment scores")
    #print(allignment_scores)
    sorted_result = sorted(
        [[weight, score, input_var] for weight, score, input_var 
         in zip(weights.flatten(), allignment_scores.flatten(), range(weights.flatten().shape[0]))],
        key=lambda x: x[1]
    )

    #print("the minus weight is {}".format(minus_weight))
    #print("the count is {}".format(count))
    

In [ ]:
arr = np.array([
    [
        [
            [1, 2, 3],
            [4, 5, 6]
        ],
        [
            [7, 8, 9],
            [10, 11, 12]
        ]
    ],
        [
        [
            [13, 14, 15],
            [16, 17, 18]
        ],
        [
            [19, 20, 21],
            [22, 23, 24]
        ]
    ]
])

np.sum(arr, axis=(1,2,3))

In [ ]:
import numpy as np

np.repeat([1,2,3], 3)

In [ ]:
import numpy as np

a = np.array([
    [
        [1, 2],
        [2, 3]
    ],
    [
        [4, 5],
        [6, 7]
    ]
])
np.sum(a, axis=2)

In [ ]:
import numpy as np

print(np.random.randint(0, 1))

b = np.arange(3)
a= np.random.choice(b, 2, replace=False)
print(a)

In [ ]:
a = np.array([1,2,3,4])
a[[1,2]] = np.array([5,6])
print(a)

In [1]:
import numpy as np

a = np.arange(18)
a = np.reshape(a, (3,2,3))
print(a.flatten())
a = np.swapaxes(a, 0, 2)
print(a.flatten())
a = np.swapaxes(a, 0, 2)
print(a.flatten())
b = np.swapaxes(a, 0, 2)
#print("a {}".format(a))
#print("b {}".format(b))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
[ 0  6 12  3  9 15  1  7 13  4 10 16  2  8 14  5 11 17]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]


In [ ]:
a = np.arange(18)
a = np.reshape(a, (3,2,3))
nudge_new = np.zeros(a.shape)
print(nudge_new.shape)
for i in [0, 1]:
    

In [2]:
class NewList(list):
    def __init__(self, individuals):
        list.__init__(self, individuals)
        self.individuals = individuals
        
    def get_best(self):
        pass
    
new_list = NewList([1,2,3])
print(new_list[:2])
print(new_list.individuals[:2])

[1, 2]
[1, 2]
